In [1]:
info = """
LangChain 1.0 - 自定义工具 (@tool 装饰器)
=========================================

本模块重点讲解：
1. 使用 @tool 装饰器创建工具（LangChain 1.0 推荐方式）
2. 工具的参数和文档字符串（docstring）的重要性
3. 测试工具
"""
print(info)


LangChain 1.0 - 自定义工具 (@tool 装饰器)

本模块重点讲解：
1. 使用 @tool 装饰器创建工具（LangChain 1.0 推荐方式）
2. 工具的参数和文档字符串（docstring）的重要性
3. 测试工具



In [3]:
import os
import sys



In [4]:
from dotenv import  load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool

from tools.weather import get_weather
from tools.calculator import calculator
from tools.web_search import web_search

In [5]:
# 加载环境变量
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if not GROQ_API_KEY or GROQ_API_KEY == "your_groq_api_key_here":
    raise ValueError(
        "\n请先在 .env 文件中设置有效的 GROQ_API_KEY\n"
        "访问 https://console.groq.com/keys 获取免费密钥"
    )

In [6]:
model = init_chat_model("groq:llama-3.3-70b-versatile", api_key=GROQ_API_KEY)

In [8]:
print("""
    示例1：使用 @tool 装饰器创建工具

    关键：
    1. 使用 @tool 装饰器
    2. 必须有 docstring（文档字符串）
    3. 参数要有类型注解
    """)


    示例1：使用 @tool 装饰器创建工具

    关键：
    1. 使用 @tool 装饰器
    2. 必须有 docstring（文档字符串）
    3. 参数要有类型注解
    


In [9]:
print("\n" + "="*70)
print("示例 1：创建第一个工具")
print("="*70)


示例 1：创建第一个工具


In [10]:
@tool
def get_current_time() -> str:
    '''获取当前时间'''
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


In [12]:
print(get_current_time.name)
print(get_current_time.description)
print(get_current_time.args)

get_current_time
获取当前时间
{}


In [14]:
result = get_current_time.invoke({})
# 被 @tool 装饰器装饰的函数会被转换为 LangChain 的 Tool
# 对象，这个对象有 .invoke() 方法。

In [15]:
print(f"\n调用结果: {result}")


调用结果: 2025-12-19 16:47:05


In [16]:
print("\n💡 关键点：")
print("  1. @tool 装饰器会自动提取函数名、docstring、参数")
print("  2. docstring 很重要！AI 用它理解工具的功能")
print("  3. 类型注解帮助 AI 理解参数类型")


💡 关键点：
  1. @tool 装饰器会自动提取函数名、docstring、参数
  2. docstring 很重要！AI 用它理解工具的功能
  3. 类型注解帮助 AI 理解参数类型


In [17]:
"""
示例2：带参数的工具

重点：参数的文档说明
"""


'\n示例2：带参数的工具\n\n重点：参数的文档说明\n'

In [18]:
print("\n" + "="*70)
print("示例 2：带参数的工具")
print("="*70)


示例 2：带参数的工具


In [23]:
print("\n查看天气工具的信息：")
print(f"名称: {get_weather.name}")
print(f"描述: {get_weather.description}")
print(f"参数: {get_weather.args}")


查看天气工具的信息：
名称: get_weather
描述: 获取指定城市的天气信息

参数:
    city: 城市名称，如"北京"、"上海"

返回:
    天气信息字符串
参数: {'city': {'title': 'City', 'type': 'string'}}


In [24]:
result1 = get_weather.invoke({"city":"北京"})
result1

'晴天，温度 15°C，空气质量良好'

In [25]:
result2 = get_weather.invoke({"city": "上海"})
print(f"上海天气: {result2}")

上海天气: 多云，温度 18°C，有轻微雾霾


In [26]:
print("\n💡 docstring 格式：")
print('''
@tool
def my_tool(param1: str) -> str:
    """
    工具的简短描述

    参数:
        param1: 参数说明

    返回:
        返回值说明
    """
''')



💡 docstring 格式：

@tool
def my_tool(param1: str) -> str:
    """
    工具的简短描述

    参数:
        param1: 参数说明

    返回:
        返回值说明
    """



In [27]:
"""
示例3：多参数工具
"""
print("\n" + "="*70)
print("示例 3：多参数工具 - 计算器")
print("="*70)



示例 3：多参数工具 - 计算器


In [28]:
print("\n计算器工具信息：")
print(f"名称: {calculator.name}")
print(f"描述: {calculator.description}")


计算器工具信息：
名称: calculator
描述: 执行基本的数学计算

参数:
    operation: 运算类型，支持 "add"(加), "subtract"(减), "multiply"(乘), "divide"(除)
    a: 第一个数字
    b: 第二个数字

返回:
    计算结果字符串


In [29]:
# 测试不同运算
print("\n测试计算：")
tests = [
    {"operation": "add", "a": 10, "b": 5},
    {"operation": "multiply", "a": 7, "b": 8},
    {"operation": "divide", "a": 20, "b": 4}
]


测试计算：


In [30]:
for test in tests:
    result = calculator.invoke(test)
    print(result)

10.0 add 5.0 = 15.0
7.0 multiply 8.0 = 56.0
20.0 divide 4.0 = 5.0


In [31]:
"""
示例4：可选参数

使用 Optional[类型] 和默认值
"""
print("\n" + "="*70)
print("示例 4：可选参数 - 搜索工具")
print("="*70)


示例 4：可选参数 - 搜索工具


In [33]:
result1 = web_search.invoke({"query": "LangChain", "num_results": 2})
result1

"搜索 'LangChain' 找到 2 条结果：\n1. LangChain官方文档\n2. LangChain GitHub仓库"

In [34]:

# 使用默认参数
print("\n使用默认参数（返回3条结果）：")
result1 = web_search.invoke({"query": "Python"})
print(result1)


使用默认参数（返回3条结果）：
搜索 'Python' 找到 3 条结果：
1. Python官方网站 - https://www.python.org
2. Python教程 - 菜鸟教程
3. Python最佳实践 - Real Python


In [35]:
"""
示例5：将工具绑定到模型

这是让 AI 使用工具的第一步
"""
print("\n" + "="*70)
print("示例 5：工具绑定到模型（预览）")
print("="*70)


示例 5：工具绑定到模型（预览）


In [36]:
model_with_tools = model.bind_tools([get_weather,get_current_time,web_search,calculator])
model_with_tools

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002226A47E900>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002226A47F770>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_weather', 'description': '获取指定城市的天气信息\n\n参数:\n    city: 城市名称，如"北京"、"上海"\n\n返回:\n    天气信息字符串', 'parameters': {'properties': {'city': {'type': 'string'}}, 'required': ['city'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'get_current_time', 'description': '获取当前时间', 'parameters': {'properties': {}, 'type': 'object'}}}, {'type': 'function', 'function': {'name'

In [39]:
response = model_with_tools.invoke("北京今天天气怎么样？")


In [40]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'gkbc1njrf', 'function': {'arguments': '{"city":"北京"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 628, 'total_tokens': 642, 'completion_time': 0.035221085, 'completion_tokens_details': None, 'prompt_time': 0.069017017, 'prompt_tokens_details': None, 'queue_time': 0.069495445, 'total_time': 0.104238102}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_43d97c5965', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b35e5-fa27-7621-af51-fd9364c1c5cd-0', tool_calls=[{'name': 'get_weather', 'args': {'city': '北京'}, 'id': 'gkbc1njrf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 628, 'output_tokens': 14, 'total_tokens': 642})

In [41]:
# 检查模型是否要求调用工具
if response.tool_calls:
    print(f"\n✅ AI 决定使用工具！")
    print(f"工具调用: {response.tool_calls}")
else:
    print(f"\nℹ️ AI 直接回答（未使用工具）")
    print(f"回复: {response.content}")

print("\n💡 下一步：")
print("  在 05_simple_agent 中，我们将学习如何让 AI 自动执行工具")



✅ AI 决定使用工具！
工具调用: [{'name': 'get_weather', 'args': {'city': '北京'}, 'id': 'gkbc1njrf', 'type': 'tool_call'}]

💡 下一步：
  在 05_simple_agent 中，我们将学习如何让 AI 自动执行工具
